# Face recognizer using pre trained model

Below is the implementation of a face recognizer which can identify the face of the person showing on a web cam. We will apply transfer learning and use pre-trained weights of VGG Face model.

# Import the requried libraries
Here we are importing all the requried libraries.

In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
import os

Using TensorFlow backend.


# Contructing the neural network model
The model here constructed is based on VGG face model.It has 22 layers and 37 deep units.
![Vgg face model](images/vgg-face-model.png)
The structure of the VGG-Face model is demonstrated below.


The VGG-Face architure can be to be understood clear with the help of following image,
![Vgg face model](images/vgg-face-architecture.jpg)



In [2]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Loading the model with pretrained weights



In [3]:



from keras.models import model_from_json

model.load_weights('C:\\Users\\Dell\\Desktop\\mlops\\Face-recognizer-using-pre-trained-model-master\\Face-recognizer-using-pre-trained-model-master/vgg_face_weights.h5')
# model.load_weights('C:/Users/Abhay/Desktop/Face-recognition-using-deep-learning-master/vgg_face_weights.h5')

## About <font color=blue>image_to_embedding</font> function        
When the model is loaded with pre trained weights, then we can create the **2622 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the VGG network to generate the embedding vector.

In [4]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (224, 224)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About <font color=blue>recognize_face</font> function
This function calculate similarity between the captured image and the images that are already been stored in the folder. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance and the cosine distance. 

Here I am using both the [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) and the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) to see for the match.Greater the Cosine similarity between two images then there is greater probability of having the image of same person. Smaller the Euclidean distance between two images then there is greater probability of having the image of same person. 

Here I am setting the thresholds for the Euclidean distance and Cosine Similarity are 0.17 and 0.70 (which can be adjusted).

In [5]:
def recognize_face(face_image, input_embeddings, model):   
    
    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    minimum_distance1 = 0
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding) * 1000
        
        a = np.matmul(embedding,input_embedding.T)
        b = np.sqrt(np.matmul(embedding,embedding.T))
        c = np.sqrt(np.matmul(input_embedding,input_embedding.T))
        cosine_dist = a / ( b * c )
        
        #print(dist1)
        #print (cosine_dist)
        
      
        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance and cosine_dist > minimum_distance1:
            minimum_distance = euclidean_distance
            minimum_distance1 = cosine_dist
            name = input_name[0]
            
    
    if minimum_distance < 0.17 and minimum_distance1 > 0.70:
        return str(name)
    else:
        return None
    

## About <font color=blue>create_input_image_embeddings</font> function
This function generates 512 dimensional image ebeddings of all the images stored in the "imagesv1" containg folders with person  names with images.We feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


In [6]:
import glob

def create_input_image_embeddings():      
    input_embeddings = {}

    for file in glob.glob("imagesv1/*"):
        for file1 in glob.glob(file+"/*"):
            person_name_image = os.path.splitext(os.path.basename(file1))[0]
            person_name = os.path.splitext(os.path.basename(file))[0]
#             print(person_name)
            image_file = cv2.imread(file1, 1)
            input_embeddings[person_name,person_name_image] = image_to_embedding(image_file, model)

    return input_embeddings

## About <font color=blue>recognize_faces_in_cam</font> function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. Here for the face detection we are using Haarcascade model.

In [7]:
def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"imagesv1"** folder.After every run this will create a new folder with name **"new user_(current date and time)"** (inorder to avoid duplication of the folder name).In this sub folder there will be 10 images of detected faces will be strored.You have to rename the folder name with the person name in order to get the correct name of person in output.You can also remove the images which are not much good from set of 10 images.

These images will be used for recognizing the the identity of the person using one shot learning.

In [8]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#  #change - to get current time and date
from datetime import datetime
date_time = datetime.now().strftime('%Y%m%d%H%M%S')

#  #change create a new folder to save images
newpath =r'C://Users//Dell//Desktop//mlops//Face-recognition-using-deep-learning-master//imagesv1//newUser'+date_time 
image_path = r'imagesv1\newUser'+date_time
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
    
count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite(image_path+"/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 10 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

In [9]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)


Euclidean distance from ('eva', '2019024_288134') is 0.1781106839189306
Euclidean distance from ('eva', '20190324_231828') is 0.1566106075188145
Euclidean distance from ('eva', '20190324_2318281') is 0.20763800421264023
Euclidean distance from ('eva', '20190324_23182812') is 0.24205006775446236
Euclidean distance from ('eva', '20190324_231828123') is 0.21677666518371552
Euclidean distance from ('eva', '20190324_2318281234') is 0.2034318313235417
Euclidean distance from ('eva', '20190324_231828134') is 0.20463815599214286
Euclidean distance from ('eva', '20190324_23188134') is 0.2037837402895093
Euclidean distance from ('eva', '20190324_288134') is 0.19144210091326386
Euclidean distance from ('eva', '20190911_16320') is 0.16661707195453346
Euclidean distance from ('eva', '20190911_163201') is 0.1530623558210209
Euclidean distance from ('eva', '2019091_163') is 0.2195322304032743
Euclidean distance from ('eva', '2019091_1632') is 0.2055938239209354
Euclidean distance from ('eva', '201909

Euclidean distance from ('eva', '2019024_288134') is 0.1760510785970837
Euclidean distance from ('eva', '20190324_231828') is 0.15501526650041342
Euclidean distance from ('eva', '20190324_2318281') is 0.24357557413168252
Euclidean distance from ('eva', '20190324_23182812') is 0.26197783881798387
Euclidean distance from ('eva', '20190324_231828123') is 0.22132198500912637
Euclidean distance from ('eva', '20190324_2318281234') is 0.19593560136854649
Euclidean distance from ('eva', '20190324_231828134') is 0.2100081619573757
Euclidean distance from ('eva', '20190324_23188134') is 0.20982603018637747
Euclidean distance from ('eva', '20190324_288134') is 0.19312532094772905
Euclidean distance from ('eva', '20190911_16320') is 0.16347180644515902
Euclidean distance from ('eva', '20190911_163201') is 0.17454964108765125
Euclidean distance from ('eva', '2019091_163') is 0.22583719692192972
Euclidean distance from ('eva', '2019091_1632') is 0.21247185941319913
Euclidean distance from ('eva', '2

Euclidean distance from ('eva', '2019024_288134') is 0.17123197903856635
Euclidean distance from ('eva', '20190324_231828') is 0.1575649657752365
Euclidean distance from ('eva', '20190324_2318281') is 0.21591191762126982
Euclidean distance from ('eva', '20190324_23182812') is 0.2400955418124795
Euclidean distance from ('eva', '20190324_231828123') is 0.21223860676400363
Euclidean distance from ('eva', '20190324_2318281234') is 0.19348380737937987
Euclidean distance from ('eva', '20190324_231828134') is 0.19846620853058994
Euclidean distance from ('eva', '20190324_23188134') is 0.19463145872578025
Euclidean distance from ('eva', '20190324_288134') is 0.18362340051680803
Euclidean distance from ('eva', '20190911_16320') is 0.1696032122708857
Euclidean distance from ('eva', '20190911_163201') is 0.15343977429438382
Euclidean distance from ('eva', '2019091_163') is 0.22895948495715857
Euclidean distance from ('eva', '2019091_1632') is 0.2161440788768232
Euclidean distance from ('eva', '201

Euclidean distance from ('eva', '2019024_288134') is 0.18905311299022287
Euclidean distance from ('eva', '20190324_231828') is 0.1858126779552549
Euclidean distance from ('eva', '20190324_2318281') is 0.24250282149296254
Euclidean distance from ('eva', '20190324_23182812') is 0.25435711722821
Euclidean distance from ('eva', '20190324_231828123') is 0.22966272081248462
Euclidean distance from ('eva', '20190324_2318281234') is 0.21245636162348092
Euclidean distance from ('eva', '20190324_231828134') is 0.21410860063042492
Euclidean distance from ('eva', '20190324_23188134') is 0.21071026276331395
Euclidean distance from ('eva', '20190324_288134') is 0.2014200872508809
Euclidean distance from ('eva', '20190911_16320') is 0.20290825341362506
Euclidean distance from ('eva', '20190911_163201') is 0.18222880316898227
Euclidean distance from ('eva', '2019091_163') is 0.2612297248560935
Euclidean distance from ('eva', '2019091_1632') is 0.2449780877213925
Euclidean distance from ('eva', '201909

Euclidean distance from ('eva', '2019024_288134') is 0.1793616102077067
Euclidean distance from ('eva', '20190324_231828') is 0.17919421952683479
Euclidean distance from ('eva', '20190324_2318281') is 0.23536906519439071
Euclidean distance from ('eva', '20190324_23182812') is 0.24998103617690504
Euclidean distance from ('eva', '20190324_231828123') is 0.2230816171504557
Euclidean distance from ('eva', '20190324_2318281234') is 0.20610839419532567
Euclidean distance from ('eva', '20190324_231828134') is 0.20572560606524348
Euclidean distance from ('eva', '20190324_23188134') is 0.2022035332629457
Euclidean distance from ('eva', '20190324_288134') is 0.19192203762941062
Euclidean distance from ('eva', '20190911_16320') is 0.1966064446605742
Euclidean distance from ('eva', '20190911_163201') is 0.17818339983932674
Euclidean distance from ('eva', '2019091_163') is 0.2544120652601123
Euclidean distance from ('eva', '2019091_1632') is 0.23838372726459056
Euclidean distance from ('eva', '2019

...

Euclidean distance from ('eva', '2019024_288134') is 0.18390349578112364
Euclidean distance from ('eva', '20190324_231828') is 0.1567386498209089
Euclidean distance from ('eva', '20190324_2318281') is 0.21499345893971622
Euclidean distance from ('eva', '20190324_23182812') is 0.22861768957227468
Euclidean distance from ('eva', '20190324_231828123') is 0.20136397506576031
Euclidean distance from ('eva', '20190324_2318281234') is 0.18702603119891137
Euclidean distance from ('eva', '20190324_231828134') is 0.1989981537917629
Euclidean distance from ('eva', '20190324_23188134') is 0.19471593259368092
Euclidean distance from ('eva', '20190324_288134') is 0.18514129624236375
Euclidean distance from ('eva', '20190911_16320') is 0.17973351350519806
Euclidean distance from ('eva', '20190911_163201') is 0.1817733864299953
Euclidean distance from ('eva', '2019091_163') is 0.23182207951322198
Euclidean distance from ('eva', '2019091_1632') is 0.21793805353809148
Euclidean distance from ('eva', '20

Euclidean distance from ('eva', '2019024_288134') is 0.17812303849495947
Euclidean distance from ('eva', '20190324_231828') is 0.154381719767116
Euclidean distance from ('eva', '20190324_2318281') is 0.22130463912617415
Euclidean distance from ('eva', '20190324_23182812') is 0.2362782252021134
Euclidean distance from ('eva', '20190324_231828123') is 0.20438416686374694
Euclidean distance from ('eva', '20190324_2318281234') is 0.18592920969240367
Euclidean distance from ('eva', '20190324_231828134') is 0.19951813737861812
Euclidean distance from ('eva', '20190324_23188134') is 0.19482815696392208
Euclidean distance from ('eva', '20190324_288134') is 0.18227174587082118
Euclidean distance from ('eva', '20190911_16320') is 0.17183726595249027
Euclidean distance from ('eva', '20190911_163201') is 0.1783389161573723
Euclidean distance from ('eva', '2019091_163') is 0.22790784714743495
Euclidean distance from ('eva', '2019091_1632') is 0.21507113706320524
Euclidean distance from ('eva', '201

Euclidean distance from ('eva', '2019024_288134') is 0.166562051163055
Euclidean distance from ('eva', '20190324_231828') is 0.1500436628703028
Euclidean distance from ('eva', '20190324_2318281') is 0.212917075259611
Euclidean distance from ('eva', '20190324_23182812') is 0.23024487018119544
Euclidean distance from ('eva', '20190324_231828123') is 0.1969383447431028
Euclidean distance from ('eva', '20190324_2318281234') is 0.1774029224179685
Euclidean distance from ('eva', '20190324_231828134') is 0.1900585921248421
Euclidean distance from ('eva', '20190324_23188134') is 0.1831131667131558
Euclidean distance from ('eva', '20190324_288134') is 0.16948924167081714
Euclidean distance from ('eva', '20190911_16320') is 0.16865383076947182
Euclidean distance from ('eva', '20190911_163201') is 0.1767688081599772
Euclidean distance from ('eva', '2019091_163') is 0.22547463595401496
Euclidean distance from ('eva', '2019091_1632') is 0.21066794579382986
Euclidean distance from ('eva', '2019091_1

Euclidean distance from ('eva', '2019024_288134') is 0.1634103391552344
Euclidean distance from ('eva', '20190324_231828') is 0.1564616395626217
Euclidean distance from ('eva', '20190324_2318281') is 0.21561933681368828
Euclidean distance from ('eva', '20190324_23182812') is 0.22785479086451232
Euclidean distance from ('eva', '20190324_231828123') is 0.19592218450270593
Euclidean distance from ('eva', '20190324_2318281234') is 0.1756655256031081
Euclidean distance from ('eva', '20190324_231828134') is 0.18659078341443092
Euclidean distance from ('eva', '20190324_23188134') is 0.17919705715030432
Euclidean distance from ('eva', '20190324_288134') is 0.16456899174954742
Euclidean distance from ('eva', '20190911_16320') is 0.17492478946223855
Euclidean distance from ('eva', '20190911_163201') is 0.18139596795663238
Euclidean distance from ('eva', '2019091_163') is 0.23055046040099114
Euclidean distance from ('eva', '2019091_1632') is 0.21596558508463204
Euclidean distance from ('eva', '20

Euclidean distance from ('eva', '2019024_288134') is 0.16567666898481548
Euclidean distance from ('eva', '20190324_231828') is 0.14494912466034293
Euclidean distance from ('eva', '20190324_2318281') is 0.2262300840811804
Euclidean distance from ('eva', '20190324_23182812') is 0.24818271049298346
Euclidean distance from ('eva', '20190324_231828123') is 0.21179964824113995
Euclidean distance from ('eva', '20190324_2318281234') is 0.185814467840828
Euclidean distance from ('eva', '20190324_231828134') is 0.19603001419454813
Euclidean distance from ('eva', '20190324_23188134') is 0.18938197172246873
Euclidean distance from ('eva', '20190324_288134') is 0.17170188948512077
Euclidean distance from ('eva', '20190911_16320') is 0.15767932927701622
Euclidean distance from ('eva', '20190911_163201') is 0.17198678688146174
Euclidean distance from ('eva', '2019091_163') is 0.21850546181667596
Euclidean distance from ('eva', '2019091_1632') is 0.2050913608400151
Euclidean distance from ('eva', '201